In [ ]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
import urllib
from datetime import datetime
import string
import pygsheets
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.golden_gate as golden_gate

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

In [ ]:
strain_sheet = gc.open_by_key(col["strains"]).worksheet()
plasmid_sheet = gc.open_by_key(col["plasmids"]).worksheet()
part_sheet = gc.open_by_key(col["parts"]).worksheet()
part_sequences_sheet = gc.open_by_key(col["parts"]).worksheet_by_title("Sequences")
oligo_sheet = gc.open_by_key(col["oligos"]).worksheet()

In [ ]:
drive_service = plasmid_sheet.client.drive.service
plasmid_folder = col["plasmid_maps"]
plasmid_maps = api.google.list_drive(drive_service, root=plasmid_folder)

# GG

In [ ]:
# gibson.assemble -> hhh

In [ ]:
def get_plib_seq(drive_service, plib_num):
    seq_file = api.google.get_drive_by_path(
        drive_service, f"pLIB{plib_num}.gbk", root=col["plasmid_maps"]
    )
    seq = api.read_sequence(
        drive_service.files().get_media(fileId=seq_file).execute().decode("utf8")
    )
    return seq

In [ ]:
seq1 = get_plib_seq(drive_service, 1)
seq2 = get_plib_seq(drive_service, 82)
seq3 = get_plib_seq(drive_service, 23)
seq4 = get_plib_seq(drive_service, 95)
seq5 = get_plib_seq(drive_service, 110)

In [ ]:
to_join = [
    (sequence.reverse_complement(seq1), Restriction.BsaI, "Name1", "promoter"),
    (sequence.reverse_complement(seq2), Restriction.BsaI, "Name2", "RBS"),
    (seq3, Restriction.BsaI, "Name3", "CDS"),
    (seq4, Restriction.BsaI, "Name4", "terminator"),
    (sequence.reverse_complement(seq5), Restriction.BsaI, "Name5", "misc_feature"),
]

assembly = golden_gate.assemble(to_join, linear=False)
assembly

In [ ]:
with open("/Users/jacob/Downloads/test3.gb", "w") as f:
    f.write(assembly.format("gb"))

# 3G

In [ ]:
command = "@gibson(@goldengate(UNS1, J23101, BCD11, mVenus, L3S3P11, UNS5), @goldengate(UNS5, J23150, CFP, BCD16, L3S2P55, UNS10))"

In [ ]:
command = "@3G(UNS1, J23101, BCD11, mVenus, L3S3P11, UNS5; UNS5, J23150, CFP, BCD16, L3S2P55, UNS10)"

In [ ]:
command = "@3G(UNS1-J23101-BCD11-mVenus-L3S3P11-UNS5, UNS5+J23150+CFP+BCD16+L3S2P55+UNS10)

# Command parsing

In [ ]:
import tatsu

In [ ]:
grammar = """@@grammar::CALC


start
    =
    expression $
    ;


expression
    =
    | left:expression op:'+' ~ right:term
    | left:expression op:'-' ~ right:term
    | term
    ;


term
    =
    | left:term op:'*' ~ right:factor
    | left:term '/' ~ right:factor
    | factor
    ;


factor
    =
    | '(' ~ @:expression ')'
    | number
    ;


number
    =
    /\d+/
    ;
"""

In [ ]:
grammar2 = """@@grammar::CALC


start
    =
    expression $
    ;


expression
    =
    | addition
    | subtraction
    | term
    ;


addition
    =
    left:expression op:'+' ~ right:term
    ;

subtraction
    =
    left:expression op:'-' ~ right:term
    ;


term
    =
    | multiplication
    | division
    | factor
    ;


multiplication
    =
    left:term op:'*' ~ right:factor
    ;


division
    =
    left:term op:'/' ~ right:factor
    ;


factor
    =
    | '(' ~ @:expression ')'
    | number
    ;


number
    =
    /\d+/
    ;"""

In [ ]:
parser = tatsu.compile(grammar2)
# ast = parser.parse('3 + 5 * ( 10 - 20 )')

In [ ]:
class CalcSemantics(object):
    def number(self, ast):
        return int(ast)

    def addition(self, ast):
        return ast.left + ast.right

    def subtraction(self, ast):
        return ast.left - ast.right

    def multiplication(self, ast):
        return ast.left * ast.right

    def division(self, ast):
        return ast.left / ast.right

In [ ]:
ast = parser.parse("3 + 5 * ( 10 - 20 )", semantics=CalcSemantics())

In [ ]:
ast

## Commands

In [ ]:
# "quoted DNA"
# oLIB1+oLIB2
# /

In [ ]:
grammar = """@@grammar::CLONING
@@whitespace :: //

start = command $ ;

ws = /\s*/ ;

command_name = '@' ~ @:/\w+/ ;

command_arglist = '(' ~ ws @+:argument ws {',' ws @+:argument ws }* ')' ;

command = command_name:command_name arguments:command_arglist ;

argument
    =
    | quoted_string
    | command
    | float
    | int
    | assembly
    | name
    ;

float = float:/\d+\.\d+/ ;

int = int:/\d+/ ;

name = name:/\w+/ ;

assembly = assembly+:name {'-' ~ assembly+:name}+ ;

quoted_string = '"' ~ quoted_string:/[^"]*/ '"' ;

"""

In [ ]:
parser = tatsu.compile(grammar)
ast = parser.parse(
    '@foo(blah, blah2, blah3-blah4, @bar(x1, x2, 3, "a", " AAtg ", 5, 8.7))'
)
ast

In [ ]:
def goldengate(*args):
    return args


def gibson(*args):
    return args


def threeg(*args):
    return args


commands = {"GG": goldengate, "Gib": gibson, "3G": threeg}

In [ ]:
command = (
    "@3G(UNS1-J23101-BCD11-mVenus-L3S3P11-UNS5, UNS5-J23150-CFP-BCD16-L3S2P55-UNS10)"
)
# command = "@Gib(@GG(UNS1, J23101, BCD11, mVenus, L3S3P11, UNS5), @GG(UNS5, J23150, CFP, BCD16, L3S2P55, UNS10))"

In [ ]:
ast = parser.parse(command)
ast

In [ ]:
class CloningCommandSemantics(object):
    def command(self, ast):
        if ast.command_name not in commands:
            raise tatsu.semantics.SemanticError(
                "command must be one of: {}".format(
                    ", ".join([f"@{k}" for k in commands.keys()])
                )
            )
        command = commands[ast.command_name]
        return command(ast.arguments)

    def int_(self, ast):
        return int(s)

    def float_(self, ast):
        return float(s)

    def name(self, ast):
        return ast.name

    def assembly(self, ast):
        return ast.assembly

    def default_(self, ast):
        print("foo")


parser.parse(command, semantics=CloningCommandSemantics())